In [9]:
import os
import toml
import pandas as pd
from utils.config_helper import update_nested_toml, load_config

breast
lung
prostate
stomach
rectal

In [27]:
TYPE = input("Enter the type of the config file: ")
CONFIG_PATH = f"../config/{TYPE}.toml"
config = load_config(CONFIG_PATH)

In [9]:
def inspect_nan(df, name):
    print(df[pd.isna(df[name])])

In [19]:
beta_file_number = config["init"]["hyper"]["beta_file_number"]
test_ratio = config["init"]["hyper"]["test_ratio"]
seed = config["init"]["hyper"]["splitting_seed"]
normal_number_0 = config["init"]["hyper"]["normal_number_0"]
if beta_file_number == 2:
    normal_number_1 = config["init"]["hyper"]["normal_number_1"]
data_source = config["init"]["hyper"]["data_source"]
is_columns_duplicated = config["init"]["hyper"]["is_columns_duplicated"]
if data_source == "GDC_stomach_GSE99553":  # god forgive me
    is_columns_duplicated_1 = config["init"]["hyper"]["is_columns_duplicated_1"]
is_oversample = config["init"]["hyper"]["is_oversample"]

In [20]:
if is_oversample:
    print("train on oversampled dataset.")
    trainOutPath = f"../{TYPE}/result/{data_source}/train{int(100-test_ratio*100)}_oversample"
else:
    print("train on original dataset.")
    trainOutPath = f"../{TYPE}/result/{data_source}/train{int(100-test_ratio*100)}"
testOutPath = f"../{TYPE}/result/{data_source}/test{int(test_ratio*100)}"

train on original dataset.


### Section. 0 Merge and Split Champ Data
- setting is_oversample = 0 to make sure three datasets are stored properly

- file paths
  - {TYPE}/result/{data_source}/test20/all_beta_normalized_1.csv
  - {TYPE}/result/{data_source}/train80/all_beta_normalized_0.csv
  - {TYPE}/result/{data_source}/train80_oversample/all_beta_normalized_0_oversample.csv


#### 0.1 Merge Dataset (if possible)

In [12]:
df0 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_0.csv")

In [42]:
if beta_file_number == 2:
    df1 = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/all_beta_normalized_1.csv")

In [13]:
# DEBUG
df0
# END

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104
0,cg00000957,0.840645,0.840645,0.838333,0.838333,0.841180,0.841180,0.854801,0.854801,0.826240,...,0.858369,0.858369,0.847875,0.847875,0.881325,0.881325,0.845785,0.845785,0.879158,0.879158
1,cg00001349,0.868204,0.868204,0.875523,0.875523,0.708645,0.708645,0.763812,0.763812,0.618374,...,0.879759,0.879759,0.759153,0.759153,0.835826,0.835826,0.842740,0.842740,0.862857,0.862857
2,cg00001583,0.106897,0.106897,0.520205,0.520205,0.113957,0.113957,0.213113,0.213113,0.063247,...,0.790970,0.790970,0.100963,0.100963,0.247293,0.247293,0.642551,0.642551,0.709084,0.709084
3,cg00002028,0.026257,0.026257,0.034299,0.034299,0.030409,0.030409,0.016650,0.016650,0.026056,...,0.021136,0.021136,0.032977,0.032977,0.025518,0.025518,0.045711,0.045711,0.035661,0.035661
4,cg00002719,0.029272,0.029272,0.490811,0.490811,0.052727,0.052727,0.045307,0.045307,0.035076,...,0.713871,0.713871,0.542124,0.542124,0.137699,0.137699,0.621165,0.621165,0.483589,0.483589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373335,cg27656573,0.972378,0.972378,0.952102,0.952102,0.956166,0.956166,0.971104,0.971104,0.961684,...,0.948461,0.948461,0.943028,0.943028,0.969645,0.969645,0.950949,0.950949,0.963958,0.963958
373336,cg27657363,0.969164,0.969164,0.931059,0.931059,0.949989,0.949989,0.952977,0.952977,0.942600,...,0.936938,0.936938,0.931106,0.931106,0.935278,0.935278,0.939360,0.939360,0.953841,0.953841
373337,cg27657537,0.131392,0.131392,0.282966,0.282966,0.080608,0.080608,0.259545,0.259545,0.103994,...,0.445348,0.445348,0.592364,0.592364,0.293623,0.293623,0.538090,0.538090,0.671126,0.671126
373338,cg27662611,0.033896,0.033896,0.033763,0.033763,0.050093,0.050093,0.041935,0.041935,0.040316,...,0.029657,0.029657,0.031203,0.031203,0.031619,0.031619,0.035420,0.035420,0.055974,0.055974


In [ ]:
# DEBUG
df1
# END

In [14]:
# potential feature loss
if beta_file_number == 2:
    feature_name_0 = df0.iloc[:, 0].tolist()
    feature_name_1 = df1.iloc[:, 0].tolist()

    feature_name = list(set(feature_name_0).intersection(feature_name_1))
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name_0)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_1", len(feature_name_1)
    )
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_intersection", len(feature_name)
    )
elif beta_file_number == 1:
    feature_name = df0.iloc[:, 0].tolist()
    update_nested_toml(
        "preprocess.merge_and_split", "feature_size_0", len(feature_name)
    )

In [46]:
if beta_file_number == 2:
    df0_join = df0[df0.iloc[:, 0].isin(feature_name)]
    df1_join = df1[df1.iloc[:, 0].isin(feature_name)]

In [15]:
if beta_file_number == 2:
    df0_join = df0_join.iloc[:, 1::is_columns_duplicated]
    if data_source == "GDC_stomach_GSE99553":  # god forgive me
        df1_join = df1_join.iloc[:, 1::is_columns_duplicated_1]
    else:
        df1_join = df1_join.iloc[:, 1::is_columns_duplicated]
    df0_join.reset_index(drop=True, inplace=True)
    df1_join.reset_index(drop=True, inplace=True)
    df0_join_normal = df0_join.iloc[:, :normal_number_0]
    df0_join_tumor = df0_join.iloc[:, normal_number_0:]
    df1_join_normal = df1_join.iloc[:, :normal_number_1]
    df1_join_tumor = df1_join.iloc[:, normal_number_1:]
elif beta_file_number == 1:
    df0_join = df0.iloc[:, 1::is_columns_duplicated]

In [48]:
if beta_file_number == 2:
    df_normal = pd.concat([df0_join_normal, df1_join_normal], axis=1)
    df_tumor = pd.concat([df0_join_tumor, df1_join_tumor], axis=1)

In [16]:
# drop those samples with missing value
# note: could use padding or other methods to fill the missing value

if beta_file_number == 2:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_dfc_shape", df_tumor.shape
    )
    df_normal.dropna(inplace=True, axis=1)
    df_tumor.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfn_shape", df_normal.shape
    )
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_dfc_shape", df_tumor.shape
    )
elif beta_file_number == 1:
    update_nested_toml(
        "preprocess.merge_and_split", "Before_dropna_df_shape", df0_join.shape
    )
    df0_join.dropna(inplace=True, axis=1)
    update_nested_toml(
        "preprocess.merge_and_split", "After_dropna_df_shape", df0_join.shape
    )

In [17]:
# Combine the normal and tumor data
if beta_file_number == 2:
    X = pd.concat([df_normal, df_tumor], axis=1).T
    y = [0] * df_normal.shape[1] + [1] * df_tumor.shape[1]
elif beta_file_number == 1:
    X = df0_join.T
    y = [0] * normal_number_0 + [1] * (df0_join.shape[1] - normal_number_0)


#### 0.2 Split Dataset

In [18]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_ratio, random_state=seed
)

In [19]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,373330,373331,373332,373333,373334,373335,373336,373337,373338,373339
777,0.874194,0.898549,0.398857,0.020607,0.050160,0.760982,0.011427,0.215980,0.946351,0.010709,...,0.859529,0.065364,0.068077,0.948688,0.394797,0.964447,0.956472,0.398151,0.054754,0.807358
1033,0.854382,0.784828,0.218870,0.035629,0.248093,0.506881,0.029541,0.254958,0.952804,0.026908,...,0.537996,0.042060,0.060707,0.940770,0.402364,0.918125,0.900551,0.279611,0.050552,0.844948
421,0.847503,0.843355,0.352496,0.013120,0.164024,0.491313,0.004611,0.207065,0.964780,0.031033,...,0.617928,0.012982,0.047191,0.964919,0.723513,0.947560,0.947550,0.230759,0.031211,0.851652
31,0.868061,0.842552,0.233753,0.016112,0.035108,0.639848,0.013337,0.310042,0.969242,0.021605,...,0.873315,0.074766,0.029090,0.970258,0.536661,0.962072,0.955651,0.374348,0.037064,0.914446
673,0.796901,0.800717,0.172407,0.039093,0.127151,0.576311,0.017989,0.202631,0.928545,0.025082,...,0.432770,0.038629,0.082540,0.934918,0.390929,0.933991,0.936911,0.207405,0.036866,0.826680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,0.866790,0.806655,0.416240,0.026609,0.292902,0.421996,0.013682,0.203901,0.961506,0.020783,...,0.653770,0.017502,0.282849,0.967401,0.549049,0.960580,0.957799,0.432242,0.043046,0.893718
213,0.719155,0.807050,0.460923,0.050882,0.579469,0.355426,0.018236,0.246163,0.950448,0.048865,...,0.843925,0.051752,0.088304,0.932057,0.373628,0.949687,0.886265,0.220012,0.052090,0.894881
541,0.853353,0.871244,0.469230,0.058028,0.505331,0.213345,0.018809,0.153391,0.942202,0.030679,...,0.792873,0.023084,0.058222,0.954512,0.331634,0.959963,0.950280,0.453540,0.020676,0.873412
871,0.874233,0.842095,0.625404,0.043325,0.574570,0.380974,0.024263,0.413974,0.967720,0.032709,...,0.821348,0.022499,0.132337,0.957993,0.544469,0.958323,0.944172,0.610849,0.064249,0.870542


In [20]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,373330,373331,373332,373333,373334,373335,373336,373337,373338,373339
1095,0.858369,0.879759,0.790970,0.021136,0.713871,0.658571,0.011863,0.248423,0.925763,0.038204,...,0.829249,0.028006,0.051122,0.954771,0.824771,0.948461,0.936938,0.445348,0.029657,0.912657
163,0.861304,0.903013,0.483905,0.030470,0.061322,0.815836,0.011458,0.137703,0.962681,0.034124,...,0.824103,0.032167,0.058357,0.963343,0.286603,0.951805,0.974713,0.286933,0.037000,0.865345
281,0.839346,0.681702,0.374366,0.050587,0.207037,0.476365,0.013547,0.273131,0.937427,0.022890,...,0.531522,0.016507,0.065529,0.957215,0.397212,0.926355,0.904491,0.306447,0.025756,0.863613
159,0.786792,0.725195,0.609813,0.017928,0.606008,0.675307,0.015704,0.211519,0.973527,0.023710,...,0.743792,0.015124,0.047552,0.973031,0.635039,0.946509,0.875143,0.480447,0.070194,0.807897
545,0.893740,0.617356,0.375981,0.044079,0.224016,0.378678,0.019420,0.186589,0.966400,0.035521,...,0.666661,0.034654,0.058494,0.954797,0.525104,0.963018,0.947219,0.450936,0.090218,0.906657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,0.823961,0.892725,0.534485,0.020759,0.849988,0.654644,0.014716,0.183644,0.969674,0.024089,...,0.906930,0.026443,0.456676,0.910563,0.391219,0.966691,0.945365,0.604613,0.018788,0.923326
437,0.830125,0.823448,0.147546,0.025986,0.147761,0.384723,0.008025,0.156083,0.965920,0.021922,...,0.571673,0.012699,0.058014,0.968958,0.686939,0.943490,0.979970,0.154611,0.027826,0.888100
353,0.870176,0.897210,0.728655,0.030020,0.763429,0.198860,0.017754,0.316493,0.963038,0.015984,...,0.846468,0.023235,0.035551,0.971799,0.428407,0.966088,0.968018,0.697369,0.038915,0.896972
769,0.857789,0.876160,0.276138,0.029392,0.128699,0.526618,0.016341,0.246346,0.936180,0.026817,...,0.399342,0.032047,0.050870,0.956981,0.329701,0.952278,0.954377,0.213356,0.039274,0.926780


#### 0.3 Oversample

In [21]:
update_nested_toml(
    "preprocess.merge_and_split", "Before_SMOTE_X_train_shape", X_train.shape
)
update_nested_toml(
    "preprocess.merge_and_split", "Before_SMOTE_y_train_shape", len(y_train)
)

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=seed)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
update_nested_toml(
    "preprocess.merge_and_split", "After_SMOTE_X_train_shape", X_resampled.shape
)
update_nested_toml(
    "preprocess.merge_and_split", "After_SMOTE_y_train_shape", len(y_resampled)
)


D:\Users\005535\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "D:\Users\005535\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [22]:
X_resampled

,0,1,2,3,4,5,6,7,8,9,...,373330,373331,373332,373333,373334,373335,373336,373337,373338,373339
0,0.874194,0.898549,0.398857,0.020607,0.050160,0.760982,0.011427,0.215980,0.946351,0.010709,...,0.859529,0.065364,0.068077,0.948688,0.394797,0.964447,0.956472,0.398151,0.054754,0.807358
1,0.854382,0.784828,0.218870,0.035629,0.248093,0.506881,0.029541,0.254958,0.952804,0.026908,...,0.537996,0.042060,0.060707,0.940770,0.402364,0.918125,0.900551,0.279611,0.050552,0.844948
2,0.847503,0.843355,0.352496,0.013120,0.164024,0.491313,0.004611,0.207065,0.964780,0.031033,...,0.617928,0.012982,0.047191,0.964919,0.723513,0.947560,0.947550,0.230759,0.031211,0.851652
3,0.868061,0.842552,0.233753,0.016112,0.035108,0.639848,0.013337,0.310042,0.969242,0.021605,...,0.873315,0.074766,0.029090,0.970258,0.536661,0.962072,0.955651,0.374348,0.037064,0.914446
4,0.796901,0.800717,0.172407,0.039093,0.127151,0.576311,0.017989,0.202631,0.928545,0.025082,...,0.432770,0.038629,0.082540,0.934918,0.390929,0.933991,0.936911,0.207405,0.036866,0.826680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,0.840390,0.720603,0.076444,0.011180,0.043671,0.580141,0.017776,0.263465,0.951016,0.033463,...,0.455161,0.019977,0.041181,0.967453,0.541578,0.956751,0.960248,0.124819,0.034598,0.848497
798,0.869196,0.820729,0.236100,0.011574,0.053275,0.673788,0.011725,0.343092,0.959367,0.018597,...,0.776069,0.048339,0.046512,0.966285,0.465367,0.963411,0.959866,0.311721,0.030701,0.899471
799,0.839838,0.789607,0.116098,0.022432,0.043107,0.561677,0.011432,0.310945,0.961374,0.022983,...,0.574184,0.025060,0.055560,0.965306,0.599410,0.970749,0.956876,0.159116,0.062245,0.854138
800,0.831794,0.725962,0.099156,0.034603,0.040768,0.441947,0.012494,0.247307,0.960900,0.029152,...,0.487354,0.022187,0.044053,0.960560,0.846388,0.961602,0.945373,0.088927,0.025888,0.869941


In [23]:
train_class_distribution_oversample = Counter(y_resampled)
train_class_distribution = Counter(y_train)
testing_class_distribution = Counter(y_test)

update_nested_toml("preprocess.merge_and_split", "training_set_samples_oversample", len(X_resampled))
update_nested_toml("preprocess.merge_and_split", "training_set_samples", len(X_train))
update_nested_toml("preprocess.merge_and_split", "testing_set_samples", len(X_test))
update_nested_toml(
    "preprocess.merge_and_split",
    "training_class_distribution_oversample",
    [train_class_distribution_oversample[0], train_class_distribution_oversample[1]]
)
update_nested_toml(
    "preprocess.merge_and_split",
    "training_class_distribution",
    [train_class_distribution[0], train_class_distribution[1]],
)
update_nested_toml(
    "preprocess.merge_and_split",
    "testing_class_distribution",
    [testing_class_distribution[0], testing_class_distribution[1]],
)

In [24]:
X_resampled.columns = feature_name
X_resampled["label"] = y_resampled
X_resampled.sort_values(by=["label"], inplace=True)
train_df_os = X_resampled.T
train_df_os.columns = range(train_df_os.shape[1])

X_train.columns = feature_name
X_train["label"] = y_train
X_train.sort_values(by=["label"], inplace=True)
train_df = X_train.T
train_df.columns = range(train_df.shape[1])

X_test.columns = feature_name
X_test["label"] = y_test
X_test.sort_values(by=["label"], inplace=True)
test_df = X_test.T
test_df.columns = range(test_df.shape[1])

train_df_os.insert(0, "Unnamed: 0", train_df_os.index)
train_df_os.reset_index(drop=True, inplace=True)
train_df.insert(0, "Unnamed: 0", train_df.index)
train_df.reset_index(drop=True, inplace=True)
test_df.insert(0, "Unnamed: 0", test_df.index)
test_df.reset_index(drop=True, inplace=True)

D:\Users\005535\AppData\Local\Temp\ipykernel_6440\3479401642.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_resampled["label"] = y_resampled


In [25]:
train_df_os

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,792,793,794,795,796,797,798,799,800,801
0,cg00000957,0.839838,0.869196,0.840390,0.867300,0.841462,0.870928,0.831764,0.856623,0.848833,...,0.871531,0.894763,0.851060,0.843531,0.903909,0.867909,0.846249,0.906945,0.831055,0.874194
1,cg00001349,0.789607,0.820729,0.720603,0.706520,0.759784,0.796039,0.734736,0.855053,0.733856,...,0.852018,0.790230,0.639505,0.642323,0.792060,0.843404,0.712787,0.919743,0.823128,0.898549
2,cg00001583,0.116098,0.236100,0.076444,0.099039,0.092811,0.241996,0.097386,0.370248,0.097112,...,0.285456,0.590495,0.372846,0.285800,0.540008,0.333855,0.390848,0.625366,0.119337,0.398857
3,cg00002028,0.022432,0.011574,0.011180,0.022356,0.022604,0.010737,0.037448,0.014392,0.033028,...,0.046700,0.028303,0.044283,0.044121,0.018964,0.061649,0.031211,0.027937,0.028180,0.020607
4,cg00002719,0.043107,0.053275,0.043671,0.037571,0.037241,0.062696,0.039086,0.413051,0.038128,...,0.624151,0.667853,0.174575,0.211430,0.699954,0.259972,0.272155,0.366524,0.504806,0.050160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.956876,0.959866,0.960248,0.944398,0.960477,0.963603,0.945100,0.963328,0.954410,...,0.896874,0.933151,0.949773,0.925802,0.962224,0.954252,0.886726,0.952598,0.948711,0.956472
373337,cg27657537,0.159116,0.311721,0.124819,0.209004,0.210694,0.260614,0.091882,0.524706,0.122102,...,0.576389,0.507485,0.407807,0.339295,0.685281,0.473129,0.381828,0.624767,0.396211,0.398151
373338,cg27662611,0.062245,0.030701,0.034598,0.051944,0.071717,0.027277,0.027135,0.040091,0.031608,...,0.067101,0.354402,0.032201,0.023581,0.050569,0.085238,0.027678,0.026869,0.048471,0.054754
373339,cg27665648,0.854138,0.899471,0.848497,0.883312,0.887948,0.886936,0.868528,0.901669,0.865901,...,0.831698,0.912109,0.896388,0.869179,0.895742,0.911328,0.782291,0.907069,0.888137,0.807358


In [26]:
# DEBUG
train_df
# END

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,431,432,433,434,435,436,437,438,439,440
0,cg00000957,0.870264,0.831980,0.895689,0.838333,0.845822,0.832966,0.869963,0.874715,0.828084,...,0.906945,0.831055,0.829687,0.854382,0.866790,0.719155,0.853353,0.874233,0.845824,0.874194
1,cg00001349,0.800196,0.670794,0.891208,0.875523,0.709133,0.715697,0.801962,0.782943,0.814481,...,0.919743,0.823128,0.802451,0.784828,0.806655,0.807050,0.871244,0.842095,0.798027,0.898549
2,cg00001583,0.238307,0.110287,0.556086,0.520205,0.072687,0.084428,0.413745,0.180205,0.115331,...,0.625366,0.119337,0.404183,0.218870,0.416240,0.460923,0.469230,0.625404,0.490934,0.398857
3,cg00002028,0.007305,0.016709,0.016043,0.034299,0.017500,0.022534,0.020385,0.029558,0.039319,...,0.027937,0.028180,0.033635,0.035629,0.026609,0.050882,0.058028,0.043325,0.039651,0.020607
4,cg00002719,0.070367,0.051344,0.046145,0.490811,0.030961,0.041747,0.012981,0.065425,0.066875,...,0.366524,0.504806,0.629115,0.248093,0.292902,0.579469,0.505331,0.574570,0.543914,0.050160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.963831,0.947092,0.968733,0.931059,0.953028,0.959128,0.958273,0.939942,0.960975,...,0.952598,0.948711,0.935802,0.900551,0.957799,0.886265,0.950280,0.944172,0.924081,0.956472
373337,cg27657537,0.252799,0.070351,0.315259,0.282966,0.202402,0.216263,0.184640,0.347057,0.183052,...,0.624767,0.396211,0.598269,0.279611,0.432242,0.220012,0.453540,0.610849,0.607388,0.398151
373338,cg27662611,0.024713,0.018048,0.033946,0.033763,0.024260,0.071756,0.043058,0.040479,0.036035,...,0.026869,0.048471,0.247071,0.050552,0.043046,0.052090,0.020676,0.064249,0.051552,0.054754
373339,cg27665648,0.885381,0.878828,0.932957,0.824831,0.875100,0.868722,0.907487,0.885974,0.911748,...,0.907069,0.888137,0.858410,0.844948,0.893718,0.894881,0.873412,0.870542,0.897935,0.807358


In [27]:
# DEBUG
test_df
# END

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,cg00000957,0.845864,0.835681,0.853063,0.841180,0.840645,0.833973,0.854618,0.820812,0.862565,...,0.838264,0.844825,0.892887,0.867261,0.852356,0.823961,0.890865,0.830125,0.870176,0.857789
1,cg00001349,0.773967,0.898174,0.720905,0.708645,0.868204,0.782644,0.775534,0.824775,0.797306,...,0.797339,0.811410,0.836854,0.855619,0.853462,0.892725,0.854986,0.823448,0.897210,0.876160
2,cg00001583,0.144976,0.036853,0.279087,0.113957,0.106897,0.134871,0.329480,0.316795,0.046604,...,0.163597,0.292316,0.196342,0.640031,0.357947,0.534485,0.588813,0.147546,0.728655,0.276138
3,cg00002028,0.027506,0.032216,0.035949,0.030409,0.026257,0.033763,0.023366,0.026860,0.027793,...,0.061315,0.051471,0.038849,0.029779,0.038226,0.020759,0.035198,0.025986,0.030020,0.029392
4,cg00002719,0.058274,0.035748,0.045736,0.052727,0.029272,0.035318,0.047342,0.044884,0.042019,...,0.095846,0.657692,0.322644,0.612964,0.447518,0.849988,0.647744,0.147761,0.763429,0.128699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373336,cg27657363,0.948103,0.949634,0.948849,0.949989,0.969164,0.926727,0.944173,0.963408,0.951192,...,0.935086,0.923680,0.953148,0.949006,0.935257,0.945365,0.961405,0.979970,0.968018,0.954377
373337,cg27657537,0.204987,0.114482,0.277116,0.080608,0.131392,0.164756,0.144737,0.315726,0.101441,...,0.365117,0.469148,0.339899,0.490546,0.407895,0.604613,0.682518,0.154611,0.697369,0.213356
373338,cg27662611,0.047586,0.076038,0.042346,0.050093,0.033896,0.034158,0.032237,0.089901,0.065757,...,0.032431,0.061379,0.072765,0.056173,0.044317,0.018788,0.010555,0.027826,0.038915,0.039274
373339,cg27665648,0.908067,0.871213,0.879406,0.869885,0.875534,0.889955,0.901831,0.910694,0.931462,...,0.905337,0.870209,0.890811,0.935343,0.873291,0.923326,0.884342,0.888100,0.896972,0.926780


In [28]:
os.makedirs(f"{trainOutPath}", exist_ok=True)
os.makedirs(
    f"{testOutPath}",
    exist_ok=True,
)
train_df_os.to_csv(f"{trainOutPath}_oversample/all_beta_normalized_0_oversample.csv", index=False)
train_df.to_csv(f"{trainOutPath}/all_beta_normalized_0.csv", index=False)
test_df.to_csv(f"{testOutPath}/all_beta_normalized_1.csv", index=False)

In [65]:
del train_df, test_df, train_df_os

#### 0.4 Upload Datasets

In [66]:
zip_filename = f"{TYPE}_beta_files.zip"

In [67]:
import zipfile

with zipfile.ZipFile(zip_filename, "w") as zipf:
    zipf.write(
        f"{trainOutPath}/all_beta_normalized_0_oversample.csv",
        arcname="all_beta_normalized_0_oversample.csv",
    )
    zipf.write(
        f"{trainOutPath}/all_beta_normalized_0.csv", arcname="all_beta_normalized_0.csv"
    )
    zipf.write(
        f"{testOutPath}/all_beta_normalized_1.csv", arcname="all_beta_normalized_1.csv"
    )

In [ ]:
from api import utils
service = utils.authenticate_drive()

In [ ]:
directory = utils.create_folder(service, TYPE)

In [ ]:
utils.run_upload_with_separate_thread(service, directory, zip_filename)

### Sec. 1 Delta Beta Calculation

In [11]:
import gdown

#### 1.1 Download

In [ ]:
url = input()

output = 'download.zip'

In [ ]:
gdown.download(url, output, quiet=False)

In [13]:
import zipfile
import shutil

with zipfile.ZipFile("download.zip", "r") as zip_ref:
    zip_ref.extractall("download")

shutil.move("download/all_beta_normalized_0.csv", f"all_beta_normalized_0.csv")
shutil.move("download/all_beta_normalized_1.csv", f"all_beta_normalized_1.csv")

os.remove("download.zip")
shutil.rmtree("download")

#### 1.2 Average Delta Beta Calculation

In [29]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [30]:
# remove outlier in terms of every column
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + IQR * 1.5
    lower_fence = Q1 - IQR * 1.5
    return upper_fence, lower_fence


def no_outlier(df):
    upper_fence, lower_fence = IQR(df)
    ddf = df[(df > lower_fence) & (df < upper_fence)]
    return ddf

In [31]:
config = load_config(CONFIG_PATH)
normal_count = config["preprocess"]["merge_and_split"]["training_class_distribution"][0]
all_beta_normalized_normal = train_df.iloc[:-1, 1 : normal_count + 1 :].T


all_beta_normalized_tumor = train_df.iloc[:-1, normal_count + 1 : :].T

In [32]:
all_beta_normalized_normal = no_outlier(all_beta_normalized_normal)
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [33]:
train_normal_avg = all_beta_normalized_normal.mean(skipna=True, axis=0)

In [34]:
train_normal_avg

0         0.853348
1         0.798057
2         0.171903
3         0.025508
4         0.053351
            ...   
373335    0.963602
373336    0.954809
373337    0.226111
373338    0.037843
373339    0.888051
Length: 373340, dtype: float64

In [35]:
all_beta_normalized_tumor

,0,1,2,3,4,5,6,7,8,9,...,373330,373331,373332,373333,373334,373335,373336,373337,373338,373339
40,0.844691,0.883599,0.055759,0.030430,0.669425,0.501841,0.012138,0.286491,0.977060,0.022602,...,0.805359,0.016103,0.048671,0.973899,0.700437,0.951508,0.961718,0.475937,0.033451,0.822936
41,0.885978,0.928476,0.541524,0.028458,0.489484,0.446004,0.011802,0.340689,0.966680,0.029579,...,0.822412,0.024706,0.052747,0.970397,0.705905,0.959115,0.960487,0.629956,0.066019,0.928393
42,0.858934,0.741403,0.085641,0.035874,0.028261,0.404488,0.011080,0.156409,0.960922,0.022939,...,0.595545,0.031568,0.045752,0.949982,0.621570,0.954068,0.942580,NaN,0.045088,0.885704
43,0.848930,0.862303,0.682962,0.022054,0.699766,0.833293,0.008373,0.117262,0.969456,0.018124,...,0.816631,0.018639,NaN,0.953250,0.245418,0.959824,0.951530,0.593755,0.029074,0.926334
44,0.886304,0.905683,0.038462,0.056411,0.815543,0.570895,0.009829,0.550945,0.962184,0.031179,...,0.861878,0.041390,0.032692,0.950695,0.565337,0.930961,0.945450,0.682258,0.034719,0.896805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,NaN,0.807050,0.460923,0.050882,0.579469,0.355426,0.018236,0.246163,0.950448,0.048865,...,0.843925,0.051752,0.088304,0.932057,0.373628,0.949687,0.886265,0.220012,0.052090,0.894881
437,0.853353,0.871244,0.469230,0.058028,0.505331,0.213345,0.018809,0.153391,0.942202,0.030679,...,0.792873,0.023084,0.058222,0.954512,0.331634,0.959963,0.950280,0.453540,0.020676,0.873412
438,0.874233,0.842095,0.625404,0.043325,0.574570,0.380974,0.024263,0.413974,0.967720,0.032709,...,0.821348,0.022499,NaN,0.957993,0.544469,0.958323,0.944172,0.610849,0.064249,0.870542
439,0.845824,0.798027,0.490934,0.039651,0.543914,0.503218,0.013511,0.338790,0.960474,0.018684,...,0.813620,0.017912,0.037178,0.957527,0.430492,0.948697,0.924081,0.607388,0.051552,0.897935


In [36]:
all_beta_normalized_tumor = all_beta_normalized_tumor.subtract(
    train_normal_avg, axis=1
)

In [37]:
all_beta_normalized_tumor = no_outlier(all_beta_normalized_tumor)

In [38]:
train_tumor_mean = all_beta_normalized_tumor.mean(skipna=True, axis=0)

In [39]:
delta_beta = pd.merge(
    train_df.iloc[:-1, :1],
    pd.DataFrame(train_tumor_mean, columns=["dbeta"]),
    left_index=True,
    right_index=True,
)
update_nested_toml("preprocess.dbeta", "delta_beta_avg", delta_beta.shape[0])

In [40]:
# print(delta_beta[pd.isna(delta_beta["dbeta"])])
# record the list of feature with dbeta being NaN
update_nested_toml(
    "preprocess.dbeta",
    "NaN_dbeta_feature",
    delta_beta.loc[pd.isna(delta_beta["dbeta"]), "Unnamed: 0"].tolist(),
)
delta_beta.dropna(inplace=True, axis=0)
update_nested_toml("preprocess.dbeta", "delta_beta_avg_remove_NaN", delta_beta.shape[0])

In [41]:
dmp = pd.read_csv(f"../{TYPE}/champ_result/{data_source}/DMP_result_0.csv")
dmp = dmp[["Unnamed: 0", "gene", "feature"]]
update_nested_toml("preprocess.dbeta", "dmp_before_dropna_shape_feature", dmp.shape[0])
dmp.dropna(inplace=True)
update_nested_toml("preprocess.dbeta", "dmp_after_dropna_shape_feature", dmp.shape[0])

In [42]:
result = pd.merge(delta_beta, dmp, on="Unnamed: 0", how="left")
update_nested_toml(
    "preprocess.dbeta", "delta_beta_avg_remove_NaN_with_gene_name", result.shape[0]
)

In [43]:
def find_max_dBeta_grouped(group):
    idx_max = group["dbeta"].abs().idxmax()
    return group.loc[idx_max]


dbeta = result.groupby("gene", as_index=False).apply(
    find_max_dBeta_grouped, include_groups=False
)

In [44]:
dbeta.columns = ["gene", "ID", "dbeta", "feature"]
dbeta = dbeta[["ID", "gene", "dbeta", "feature"]]
# DEBUG
dbeta
# END

,ID,gene,dbeta,feature
0,cg03630821,A1BG,0.122476,Body
1,cg27394794,A1CF,-0.320159,Body
2,cg07196505,A2BP1,-0.407458,Body
3,cg00336946,A2LD1,-0.217054,TSS1500
4,cg00134295,A2M,-0.176973,TSS1500
...,...,...,...,...
18647,cg00692173,ZYG11B,0.166651,Body
18648,cg18074834,ZYX,0.046360,Body
18649,cg16463044,ZZEF1,-0.141835,Body
18650,cg22221847,ZZZ3,-0.028762,3'UTR


In [ ]:
# comorbidity = pd.read_csv(
#     "../external_result/matchgene174_single_3Y10__OR2.txt", sep="\t", header=None
# )
# dbeta = dbeta[
#     dbeta["gene"].isin(comorbidity[0])
# ]

# result_max_per_gene_single

In [45]:
dbeta["dbeta"] = dbeta["dbeta"].apply(lambda x: round(x, 6))
dbeta.to_csv(f"{trainOutPath}/dbeta.csv", index=False)

### Sec. 2 Filter Genes by Average Delta Beta Values
1. filter genes by dbeta values
3. filter genes by TSS position
4. plot distribution of dbeta values
5. plot PCA for normal and tumor


In [46]:
# dbeta = pd.read_csv(f"{trainOutPath}/dbeta.csv")

#### 2.1 Filtering TSS

In [47]:
TSS = dbeta[dbeta["feature"].str.contains("TSS")]

In [48]:
TSS.to_csv(f"{trainOutPath}/dbeta_TSS.csv", index=False)

#### 2.2 Thresholding

In [49]:
threshold = 1
TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
while True:
    TSS_threshold = TSS[abs(TSS["dbeta"]) > threshold]
    count = TSS_threshold.shape[0]
    if (
        config["preprocess"]["filtering"]["hyper"]["avg_dbeta_lower_bound"]
        <= count
        <= config["preprocess"]["filtering"]["hyper"]["avg_dbeta_upper_bound"]
    ):
        break
    threshold -= 0.01
threshold = round(threshold, 2)
update_nested_toml("preprocess.filtering", "threshold", threshold)

In [50]:
TSS_threshold.to_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv", index=False)

#### 2.3 Visualization

In [51]:
# DEBUG
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(TSS_threshold["dbeta"])
plt.xlabel("delta Beta value")
plt.title("Density plot of delta Beta value")
# save the plot
plt.savefig(f"{trainOutPath}/dbeta_TSS_{threshold}.png")
plt.close()
# END

In [ ]:
# train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")

In [52]:
normal_count = (train_df.iloc[-1, 1:] == 0).sum()
df_gene = train_df.iloc[:-1, :]
df_gene = df_gene[df_gene[df_gene.columns[0]].isin(dbeta["ID"])]
X = df_gene.iloc[:, 1:].reset_index(drop=True).T
y = [0 if i < normal_count else 1 for i in range(X.shape[0])]
# DEBUG
print(f"X shape: {X.shape}")
print(f"y shape: {len(y)}")
# END

X shape: (441, 18652)
y shape: 441


In [65]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

df = pd.DataFrame(
    {
        "Principal Component 1": X_pca[:, 0],
        "Principal Component 2": X_pca[:, 1],
        "Principal Component 3": X_pca[:, 2],
        "Class": y,
    }
)
print(df.shape)
fig = px.scatter_3d(
    df,
    x="Principal Component 1",
    y="Principal Component 2",
    z="Principal Component 3",
    color="Class",
    title="PCA of Dataset",
    color_continuous_scale="Viridis",
)

fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3",
    )
)

# fig.show()

fig.write_html(f"{trainOutPath}/preprocess_filtering_pca.html")

# open in browser
# save the plot

(441, 4)


### Sec. 3 Machine Learning

#### 3.1 Preparation

In [54]:
config = load_config(CONFIG_PATH)
threshold = config["preprocess"]["filtering"]["threshold"]
TSS_threshold = pd.read_csv(f"{trainOutPath}/dbeta_TSS_{threshold}.csv")

In [ ]:
# TSS_threshold_hyper = TSS_threshold[TSS_threshold["dbeta"] > 0]
# # DEBUG
# TSS_threshold_hyper
# # END

In [55]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")
test_df = pd.read_csv(f"{testOutPath}/all_beta_normalized_1.csv")

In [56]:
X_train = train_df[train_df["Unnamed: 0"].isin(TSS_threshold["ID"])]
X_test = test_df[test_df["Unnamed: 0"].isin(TSS_threshold["ID"])]

In [57]:
X_train = X_train.iloc[:, 1:].T.values.tolist()
X_test = X_test.iloc[:, 1:].T.values.tolist()

In [58]:
config = load_config(CONFIG_PATH)
normal_count_train = config["preprocess"]["merge_and_split"][
    "training_class_distribution"
][0]

normal_count_test = config["preprocess"]["merge_and_split"][
    "testing_class_distribution"
][0]
y_train = [0 if i < normal_count_train else 1 for i in range(len(X_train))]
y_test = [0 if i < normal_count_test else 1 for i in range(len(X_test))]

#### 3.2 Training

In [61]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from utils.train_helper import set_parameters, append_to_file, record

In [66]:
selection_models = {
    "XGBoost": XGBClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(kernel="linear", random_state=42),
}

param_xgb = {
    "learning_rate": [0.001, 0.01, 0.1],
    "n_estimators": [50, 100, 200],
    "max_depth": [2, 3, 4],
    "subsample": [0.6, 0.7, 0.8],
    "colsample_bytree": [0.6, 0.7, 0.8],
    "min_child_weight": [40, 50, 60],
}

param_rf = {
    "n_estimators": [50, 100, 150, 200],
    "min_samples_split": [80, 90],
    "min_samples_leaf": [80, 90],
    "max_features": [5, 10, 20],
}

param_svm = {"kernel": ["rbf", "poly", "sigmoid"], "C": [50, 100, 200]}

param_dt = {
    "min_samples_split": [70, 80, 90],
    "min_samples_leaf": [70, 80, 90],
    "max_features": [5, 10, 20],
}

xgb_model = set_parameters(XGBClassifier(random_state=42), param_xgb)
rf_model = set_parameters(RandomForestClassifier(random_state=42), param_rf)
svm_model = set_parameters(SVC(random_state=42), param_svm)
dt_model = set_parameters(DecisionTreeClassifier(random_state=42), param_dt)

train_models = {
    "XGBoost": xgb_model,
    "Random Forest": rf_model,
    "Decision Tree": dt_model,
    "SVM": svm_model,
}


In [67]:
for selection_model_name, selection_model in selection_models.items():
    for feature_count in range(20, 81, 5):
        rfe = RFE(estimator=selection_model, n_features_to_select=feature_count)
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_test_rfe = rfe.transform(X_test)
        selected_feature_names = (
            pd.DataFrame(TSS_threshold.iloc[rfe.support_, 1])
            .reset_index(drop=True)
            .T
        )

        label = f"{selection_model_name}_{feature_count}"

        selected_feature_names.insert(0, "selection_model_name", label)

        append_to_file(
            f"{trainOutPath}/selected_feature_names.csv", selected_feature_names
        )

        for train_model_name, train_model in train_models.items():
            train_model.fit(X_train_rfe, y_train)
            best_params = train_model.best_params_

            best_score = train_model.best_score_

            record(
                train_model.best_estimator_,
                X_train_rfe,
                y_train,
                selection_model_name,
                train_model_name,
                feature_count,
                best_score,
                best_params,
                f"{trainOutPath}",
            )
            record(
                train_model.best_estimator_,
                X_test_rfe,
                y_test,
                selection_model_name,
                train_model_name,
                feature_count,
                best_score,
                best_params,
                f"{testOutPath}",
            )

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 48 candidates, totalling 240 fits


D:\Users\005535\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2892: RuntimeWarning:

invalid value encountered in cast



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Fitting 

#### 3.3 Visualization

In [68]:
fpr_tpr_train = pd.read_csv(f"{trainOutPath}/fpr_tpr.csv")
fpr_tpr_test = pd.read_csv(f"{testOutPath}/fpr_tpr.csv")
rfe_train = pd.read_csv(f"{trainOutPath}/rfe.csv")
rfe_test = pd.read_csv(f"{testOutPath}/rfe.csv")

In [69]:
rfe_j = pd.merge(rfe_train, rfe_test, on=["selection_model", "train_model", "features"], suffixes=('_train', '_test'))
fpr_tpr_j = pd.merge(fpr_tpr_train, fpr_tpr_test, on=["selection_model", "train_model", "features"], suffixes=('_train', '_test'))
J = pd.merge(rfe_j, fpr_tpr_j, on=["selection_model", "train_model", "features", "AUC_train", "AUC_test"])

In [70]:
import ast

J["fpr_train"] = J["fpr_train"].apply(ast.literal_eval)
J["tpr_train"] = J["tpr_train"].apply(ast.literal_eval)
J["fpr_test"] = J["fpr_test"].apply(ast.literal_eval)
J["tpr_test"] = J["tpr_test"].apply(ast.literal_eval)

In [71]:
from utils.painter import plot_roc_curve, create_performance_barchart

In [72]:
J['accuracy_diff'] = J['accuracy_train'] - J['accuracy_test']
J['recall_diff'] = J['recall_train'] - J['recall_test']
J['f1_score_diff'] = J['f1_score_train'] - J['f1_score_test']
J['AUC_diff'] = J['AUC_train'] - J['AUC_test']
J['MCC_diff'] = J['MCC_train'] - J['MCC_test']

In [73]:
# tweakable width and height
plot_roc_curve(J, "ROC Curves on Training Set", f"{trainOutPath}/roc_train.html")

ROC curve saved to ../prostate/result/GDC_prostate_tissue/train80/roc_train.html


In [74]:
# tweakable width and height
plot_roc_curve(J, "ROC Curves on Testing Set", f"{testOutPath}/roc_test.html", mode="test")

ROC curve saved to ../prostate/result/GDC_prostate_tissue/test20/roc_test.html


#### 3.4 Filter Combinations of "selection_model", "training_model", and "features"

In [75]:
# eliminate the those with abs accuracy difference greater than 0.1
J = J[abs(J["accuracy_diff"]) < 0.1]

In [76]:
# plot difference
performance_metrics = ['accuracy_diff', 'recall_diff', 'f1_score_diff', 'AUC_diff', 'MCC_diff']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(f"{trainOutPath}/performance_diff_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_diff": "blue",
    "recall_diff": "red",
    "f1_score_diff": "green",
    "AUC_diff": "purple",
    "MCC_diff": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{trainOutPath}/performance_diff_grouped_by_train_model.html",
    title="Grouped Performance Difference between Training and Testing Set",
    x_axis_label="Performance Difference (Training - Testing)",
    y_axis_label="Train Model",
    orientation="h",
)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/train80/performance_diff_grouped_by_train_model.html


In [77]:
J = J[["selection_model", "train_model", "features", "accuracy_test", "recall_test", "f1_score_test", "AUC_test", "MCC_test"]]

In [78]:
# group by train_model, for each train_model, calculate the mean of each performance metric
performance_metrics = ['accuracy_test', 'recall_test',
                       'f1_score_test', 'AUC_test', 'MCC_test']
ground_by_train_model = J.groupby('train_model')[performance_metrics].mean()
ground_by_train_model['train_model'] = ground_by_train_model.index
ground_by_train_model.to_csv(
    f"{trainOutPath}/performance_metrics_grouped_by_train_model.csv", index=False)
color_mapping = {
    "accuracy_test": "blue",
    "recall_test": "red",
    "f1_score_test": "green",
    "AUC_test": "purple",
    "MCC_test": "orange",
}
create_performance_barchart(
    df=ground_by_train_model,
    color_mapping=color_mapping,
    metric="train_model",
    out_path=f"{trainOutPath}/performance_metrics_grouped_by_train_model.html",
    title="Grouped Performance Metrics by Train Model",
    x_axis_label="Performance",
    y_axis_label="Train Model",
    orientation="h",
)
best_train_model = ground_by_train_model['MCC_test'].idxmax()
print(f"Best train model: {best_train_model}")
ground_by_feature = J[J['train_model'] == best_train_model].groupby('features')[
    performance_metrics].mean()
ground_by_feature['features'] = ground_by_feature.index
ground_by_feature.to_csv(
    f"{trainOutPath}/performance_metrics_grouped_by_feature.csv", index=False)
create_performance_barchart(
    df=ground_by_feature,
    color_mapping=color_mapping,
    metric="features",
    out_path=f"{trainOutPath}/performance_metrics_grouped_by_feature.html",
    title="Grouped Performance Metrics by Feature",
    x_axis_label="Performance",
    y_axis_label="Feature",
    orientation="h",
)
best_num_of_feature = ground_by_feature['MCC_test'].idxmax()
print(f"Best number of feature: {best_num_of_feature}")
best_performance_records = J[(J['train_model'] == best_train_model) & (
    J['features'] == best_num_of_feature)]
best_performance_records.to_csv(g
    f"{trainOutPath}/best_performance_records.csv", index=False)

Performance difference saved to ../prostate/result/GDC_prostate_tissue/train80/performance_metrics_grouped_by_train_model.html
Best train model: SVM
Performance difference saved to ../prostate/result/GDC_prostate_tissue/train80/performance_metrics_grouped_by_feature.html
Best number of feature: 70


In [79]:
first = True
gene_set = set()
with open(f"{trainOutPath}/selected_feature_names.csv", "r") as f:
    for line in f:
        if first:
            first = False
            continue
        if line.split(",")[0].endswith(str(best_num_of_feature)):
            selected_feature_names = line.split(",")[1:]
            selected_feature_names[-1] = selected_feature_names[-1].strip()
            gene_set.update(selected_feature_names)

gene_list = pd.DataFrame(list(gene_set), columns=["gene"])
gene_list.to_csv(f"{trainOutPath}/selected_feature_set.csv", index=False)

### Sec. 4 Clustering

#### 4.1 load data

In [6]:
import pandas as pd
import numpy as np
from utils.clustering_helper import hierarchical_clustering, check_distance_matrix

In [28]:
dbeta_path = config["clustering"]["hyper"]["dbeta_file"]
gene_set_file = config["clustering"]["hyper"]["gene_set_file"]
bp_file = config["clustering"]["hyper"]["bp_file"]
cc_file = config["clustering"]["hyper"]["cc_file"]
mf_file = config["clustering"]["hyper"]["mf_file"]
terms_count_file = config["clustering"]["hyper"]["terms_count_file"]

In [29]:
dbeta = pd.read_csv(f"{trainOutPath}/{dbeta_path}.csv", index_col=0)
gene_set = pd.read_csv(f"{trainOutPath}/{gene_set_file}.csv", index_col=0)
distance_matrix_bp = pd.read_csv(f"{trainOutPath}/{bp_file}.csv", index_col=0)
distance_matrix_cc = pd.read_csv(f"{trainOutPath}/{cc_file}.csv", index_col=0)
distance_matrix_mf = pd.read_csv(f"{trainOutPath}/{mf_file}.csv", index_col=0)
terms_count = pd.read_csv(f"{trainOutPath}/{terms_count_file}.csv", index_col=0)

In [30]:
# replace NaN with 0
distance_matrix_bp = distance_matrix_bp.fillna(0)
distance_matrix_cc = distance_matrix_cc.fillna(0)
distance_matrix_mf = distance_matrix_mf.fillna(0)

In [31]:
# reindex distance matrix
index_bp = distance_matrix_bp.index
index_cc = distance_matrix_cc.index
index_mf = distance_matrix_mf.index
index = index_bp.union(index_cc).union(index_mf)
distance_matrix_bp_ = distance_matrix_bp.reindex(index=index, columns=index, fill_value=0)
distance_matrix_cc_ = distance_matrix_cc.reindex(index=index, columns=index, fill_value=0)
distance_matrix_mf_ = distance_matrix_mf.reindex(index=index, columns=index, fill_value=0)

In [32]:
# make a array of distance matrix for each ontology
distance_matrix = []

distance_matrix.append(distance_matrix_bp_)
distance_matrix.append(distance_matrix_cc_)
distance_matrix.append(distance_matrix_mf_)

#### 4.2 Weighted Sum

In [33]:
weight = [count for count in terms_count["count"]]
weight = weight / np.sum(weight)
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])

valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks

weight_sums = valid_weights.sum(axis=0)

normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)


weighted_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)

weighted_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF48,ZNF577,ZNF648,ZNF655,ZP4
ACE,0.000000,0.690013,0.805765,0.637504,0.597412,0.660149,0.724040,0.056375,0.751434,0.697314,...,0.673222,0.721243,0.728313,0.706338,0.729743,0.785282,0.617973,0.788172,0.788893,0.745105
ACOT11,0.690013,0.000000,0.454229,0.655252,0.694421,0.635476,0.496537,0.041854,0.644318,0.510734,...,0.715572,0.705818,0.682993,0.687805,0.755394,0.752457,0.718020,0.759102,0.731587,0.739752
ACSF2,0.805765,0.454229,0.000000,0.822319,0.790916,0.756245,0.677267,0.059127,0.623949,0.519881,...,0.775754,0.796536,0.731077,0.823078,0.780517,0.713800,0.706313,0.721311,0.738220,0.805920
ADAM11,0.637504,0.655252,0.822319,0.000000,0.587065,0.704027,0.631173,0.178823,0.793147,0.756452,...,0.712262,0.746492,0.772532,0.716756,0.823593,0.852584,0.797796,0.855181,0.841362,0.761520
ADAMTS12,0.597412,0.694421,0.790916,0.587065,0.000000,0.674923,0.651679,0.056470,0.743394,0.700046,...,0.650572,0.689017,0.668338,0.633971,0.705853,0.754840,0.692266,0.756824,0.742766,0.679116


In [34]:
cluster_result_weighted = hierarchical_clustering(
    weighted_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{trainOutPath}/hierarchical_clustering_weighted_sum.png",
)

chosen number of clusters: 3


In [35]:
cluster_result_weighted.head()

,gene,cluster
0,ACE,3
1,ACOT11,3
2,ACSF2,3
3,ADAM11,3
4,ADAMTS12,3


#### 4.3 Simple average

In [36]:
weight = [1, 1, 1]
masks = np.array([~np.isnan(distance_matrix[i].values) for i in range(3)])
valid_weights = np.array([weight[i] for i in range(3)])[:, None, None] * masks
weight_sums = valid_weights.sum(axis=0)
normalized_weights = np.divide(valid_weights, weight_sums, where=weight_sums != 0)
weighted_sum = sum(
    np.nan_to_num(distance_matrix[i].values) * normalized_weights[i] for i in range(3)
)
simple_sum_dataframe = pd.DataFrame(weighted_sum, index=index, columns=index)
simple_sum_dataframe.head()

,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF48,ZNF577,ZNF648,ZNF655,ZP4
ACE,0.000000,0.603333,0.727000,0.558000,0.518333,0.577000,0.597333,0.198000,0.651667,0.574000,...,0.598667,0.497000,0.645000,0.631667,0.674333,0.645667,0.563000,0.650000,0.683333,0.606667
ACOT11,0.603333,0.000000,0.442000,0.670667,0.635000,0.581667,0.490333,0.147000,0.476333,0.441667,...,0.619667,0.593000,0.559333,0.578333,0.635667,0.636000,0.625333,0.646333,0.571000,0.659333
ACSF2,0.727000,0.442000,0.000000,0.802667,0.739000,0.689333,0.605333,0.207667,0.527667,0.568000,...,0.700000,0.740333,0.634333,0.723000,0.684000,0.569333,0.573667,0.581333,0.598667,0.738000
ADAM11,0.558000,0.670667,0.802667,0.000000,0.509333,0.659333,0.581667,0.367667,0.761667,0.683667,...,0.695333,0.714667,0.729000,0.699667,0.786333,0.811667,0.787333,0.816667,0.781667,0.614667
ADAMTS12,0.518333,0.635000,0.739000,0.509333,0.000000,0.614667,0.563667,0.198333,0.665333,0.620667,...,0.576667,0.507667,0.613000,0.529333,0.637667,0.665333,0.636000,0.669000,0.640333,0.481000


In [37]:
cluster_result_simple = hierarchical_clustering(
    simple_sum_dataframe,
    range_min=2,
    range_max=4,
    cluster_number=3,
    out_path=f"{trainOutPath}/hierarchical_clustering_simple_sum.png",
)

chosen number of clusters: 3


In [38]:
cluster_result_simple.head()

,gene,cluster
0,ACE,3
1,ACOT11,3
2,ACSF2,3
3,ADAM11,3
4,ADAMTS12,3


#### 4.4 Consensus clustering 

In [39]:
cluster_bp = hierarchical_clustering(
    distance_matrix_bp, out_path=f"{trainOutPath}/hierarchical_clustering_bp.png"
)
cluster_cc = hierarchical_clustering(
    distance_matrix_cc, out_path=f"{trainOutPath}/hierarchical_clustering_cc.png"
)
cluster_mf = hierarchical_clustering(
    distance_matrix_mf, out_path=f"{trainOutPath}/hierarchical_clustering_mf.png"
)

Best number of clusters: 31
Best number of clusters: 3
Best number of clusters: 2


In [40]:
cluster_bp.columns = ["gene", "cluster_bp"]
cluster_cc.columns = ["gene", "cluster_cc"]
cluster_mf.columns = ["gene", "cluster_mf"]
cluster_bp_cc = pd.merge(cluster_bp, cluster_cc, on="gene", how="outer")
cluster_go = pd.merge(cluster_bp_cc, cluster_mf, on="gene", how="outer")
cluster_go = cluster_go.fillna(-1)
print(cluster_go.shape)
cluster_go.head()

(324, 4)


,gene,cluster_bp,cluster_cc,cluster_mf
0,ACE,7.0,2.0,2.0
1,ACOT11,5.0,1.0,2.0
2,ACSF2,5.0,1.0,2.0
3,ADAM11,21.0,2.0,2.0
4,ADAMTS12,7.0,3.0,2.0


In [41]:
num_genes = cluster_go.shape[0]
consensus_matrix = np.zeros((num_genes, num_genes))

for i in range(num_genes):
    for j in range(i, num_genes):
        if cluster_go.iloc[i]["cluster_bp"] == cluster_go.iloc[j]["cluster_bp"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_cc"] == cluster_go.iloc[j]["cluster_cc"]:
            consensus_matrix[i][j] += 1

        if cluster_go.iloc[i]["cluster_mf"] == cluster_go.iloc[j]["cluster_mf"]:
            consensus_matrix[i][j] += 1


consensus_matrix = pd.DataFrame(
    consensus_matrix, index=cluster_go["gene"], columns=cluster_go["gene"]
)
consensus_matrix += consensus_matrix.T


distance_matrix_consensus = 1 - consensus_matrix / 3
np.fill_diagonal(distance_matrix_consensus.values, 0)


distance_matrix_consensus.head()

gene,ACE,ACOT11,ACSF2,ADAM11,ADAMTS12,ADCY1,ADCY4,AFAP1L1,ALDH1L1,ALOX12,...,WDR69,WFDC5,WIPF1,WNT16,ZFP42,ZNF48,ZNF577,ZNF648,ZNF655,ZP4
gene,,,,,,,,,,,,,,,,,,,,,
ACE,0.000000,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
ACOT11,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.666667,0.333333,0.333333,0.000000,...,0.333333,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ACSF2,0.666667,0.000000,0.000000,0.666667,0.666667,0.666667,0.666667,0.333333,0.333333,0.000000,...,0.333333,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.333333,0.333333,0.666667
ADAM11,0.333333,0.666667,0.666667,0.000000,0.666667,0.666667,0.333333,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333
ADAMTS12,0.333333,0.666667,0.666667,0.666667,0.000000,0.333333,0.666667,0.666667,0.333333,0.666667,...,0.666667,0.333333,0.666667,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333


In [42]:
cluster_result_consensus = hierarchical_clustering(
    distance_matrix_consensus,
    range_min=2,
    range_max=4,
    cluster_number=4,
    out_path=f"{trainOutPath}/hierarchical_clustering_consensus.png",
)

chosen number of clusters: 4


In [43]:
cluster_result_consensus.head()

,gene,cluster
0,ACE,1
1,ACOT11,4
2,ACSF2,4
3,ADAM11,1
4,ADAMTS12,3


#### 4.5 Compare 

In [44]:
from utils.clustering_helper import hierarchical_clustering_compare

hierarchical_clustering_compare(
    [weighted_sum_dataframe, simple_sum_dataframe, distance_matrix_consensus],
    ["Weighted Average", "Simple Average", "Consensus"],
    out_path=f"{trainOutPath}/hierarchical_clustering_compare.png",
)

Best number of clusters for Weighted Average: 2
Best number of clusters for Simple Average: 5
Best number of clusters for Consensus: 5


In [45]:
dbeta["ID"] = dbeta.index

In [46]:
# column gene isin weighted_sum_dataframe
weighted_dbeta = dbeta[dbeta["gene"].isin(weighted_sum_dataframe.index)]
simple_dbeta = dbeta[dbeta["gene"].isin(simple_sum_dataframe.index)]
consensus_dbeta = dbeta[dbeta["gene"].isin(distance_matrix_consensus.index)]

In [47]:
weighted_dbeta.merge(cluster_result_weighted, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_weighted.csv", index=False
)
simple_dbeta.merge(cluster_result_simple, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_simple.csv", index=False
)
consensus_dbeta.merge(cluster_result_consensus, on="gene").to_csv(
    f"{trainOutPath}/{dbeta_path}_consensus.csv", index=False
)

### Sec. 5 Combination

#### 5.1 preparation

In [48]:
import pandas as pd

In [49]:
dbeta_path = config["combination"]["hyper"]["dbeta_file"]
dbeta = pd.read_csv(f"{trainOutPath}/{dbeta_path}.csv")
dbeta

KeyError: 'combination'

In [8]:
gene_list = pd.read_csv(f"{trainOutPath}/selected_feature_set.csv")
dbeta[dbeta["gene"].isin(gene_list["gene"])].to_csv(
    f"{trainOutPath}/{dbeta_path}_selected.csv", index=False
)

In [9]:
train_df = pd.read_csv(f"{trainOutPath}/all_beta_normalized_0.csv")
test_df = pd.read_csv(f"{testOutPath}/all_beta_normalized_1.csv")

#### 5.2 training

In [10]:
from utils.combination_helper import CombinationHelper

In [11]:
helper = CombinationHelper(
    train_df=train_df,
    test_df=test_df,
    dbeta=dbeta,
    gene_list=gene_list,
)

In [12]:
helper.setup_dbeta()
helper.setup_train_test()
helper.setup_combinations()
helper.setup_estimators()
helper.setup_grids()
helper.setup_grid_estimator()
helper.setup_grid_estimator()


In [ ]:
helper.train(
    train_folder=trainOutPath,
    test_folder=testOutPath,
    filename="combination",
    discard_overfitting=True
)